In [ ]:
import numpy as np
import math
import scipy.linalg as la
import matplotlib.pyplot as plt

In [ ]:
# def f(x): #Här skapar vi en testfunktion 
#     return x**2

# def los(x,alpha,beta,L): #Här definierar vi den exakta lösningen till ovanstående testfunktion
#     return (x**4)/12 + (beta - alpha - (L**4)/12)*x/L + alpha

def los(x, alpha, beta, L):
    return np.log(x +1) + (beta - alpha - np.log(L+1))*x/L + alpha

def f(x):
    return -1/((x+1)**2)


In [ ]:
def twopBVP(fvec, alpha, beta, L, N):
    deltaX = L/(N+1)
    T = np.eye(len(fvec))*-2 
    for i in range(len(fvec)-1): #Denna for loopen  skapar T-matrisen. Kan skrivas mer effektivt. 
        T[i+1,i] = 1
        T[i, i+1] = 1
        
    return (deltaX**2)*np.linalg.inv(T)@fvec #Detta löser ekvationen och spottar ut en vektor innehållande y(i). 

def solve(f,alpha, beta, L, N): #Tanken är att denna ska generera högerledet som man sedan stoppar in i twopBVP för att lösa. 
    deltaX = L/(N+1)
    fvec = np.zeros(((N-1), 1)) ##fvec är två element kortare än grid för den innehåller bara de inre punkterna
    grid = np.linspace(0,L,N+1)

    fvec[0] = -alpha/(deltaX**2) + f(grid[1]) #Första och sista elementet i fvec fixar vi manuellt. 
    fvec[-1] = -beta/(deltaX**2) + f(grid[-2])

    for i in range(1, N-2):  #Fyller fvec med funktionsvärdena F(x1) till F(xN-1)
        fvec[i] = f(grid[i])

    ans = np.zeros(((N+1), 1))
    ans[0] = alpha
    ans[1:N] = twopBVP(fvec, alpha, beta,  L, N)
    ans[-1] = beta

    return ans, grid #Spottar ut grid för att plotta sen 

    


In [ ]:
def errVSh(f,alpha, beta, L, N):
    Nvec = [int(np.power(2,i)) for i in range(4, 13)]
    err = np.zeros(len(Nvec))

    for i in range(0, len(Nvec)):
        y, x = solve(f, alpha, beta, L, Nvec[i])
        y_an = np.zeros(((Nvec[i]+1), 1))
        j=0
        for j in range(Nvec[i]+1):
            y_an[j] = los(x[j], alpha, beta, L)
        err[i] = np.sqrt(L/(Nvec[i]+1)) * la.norm(y-y_an) 

    deltaX=np.zeros(len(Nvec))
    for i in range(len(Nvec)):
        deltaX[i] = L / (Nvec[i] +1)
    plt.grid()
    plt.xlabel("Stepsize deltaX")
    plt.ylabel("The global error")
    plt.title("The global error as a function of stepsize")
    #plt.loglog(deltaX, err, 'r')
    return np.log(err), np.log(deltaX)

alpha, beta, L, N = 3, 8, 50, 200

x, y = errVSh(f, alpha, beta, L, N)
plt.plot(x, y)

In [ ]:
# alpha, beta, L, N = 3, -5, 50, 200



# deltaX = L/(N+1)
# y, x = solve(f, alpha, beta, L, N)

# y_an = np.zeros(((N+1), 1))
# for i in range(N+1):
#     y_an[i] = los(x[i], alpha, beta, L)

# xlog = np.log(x)
# errlog = np.log(np.abs(y-y_an))


# err = np.sqrt(deltaX) * la.norm(y-y_an)

# plt.loglog(x, err)
# plt.plot(x, y)
# plt.plot(x, y_an)
# plt.plot(x, np.zeros_like(y))
#plt.plot(xlog[0:N], errlog[0:N])